In [14]:
import random

import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from typing import List
from tqdm.auto import tqdm

In [15]:
"""标签反转 reconstruct train/val, keep test_data
concat (train, val)
-> divide by peptide-HLA pairs
"""

'标签反转 reconstruct train/val, keep test_data\nconcat (train, val)\n-> divide by peptide-HLA pairs\n'

In [16]:
train_path = '../data/AOMP_ALL_RAW/train_data_fold0.csv'
val_path = '../data/AOMP_ALL_RAW/val_data_fold0.csv'
new_train_path = '../data/AOMP_REVERSE/train.csv'
new_val_path = '../data/AOMP_REVERSE/val.csv'

In [17]:
df = pd.concat([pd.read_csv(train_path, index_col=0), pd.read_csv(val_path, index_col=0)])
df.head()

,peptide,length,HLA,label,HLA_sequence
0,FLTGTFVTA,9,HLA-A*24:02,0,YSAMYEEKVAHTDENIAYLMFHYYTWAVQAYTGY
1,HRKVMSQNF,9,HLA-B*27:05,1,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY
2,VMDKYILDN,9,HLA-B*44:03,0,YYTKYREISTNTYENTAYIRYDDYTWAVLAYLSY
3,MTFDGEVKT,9,HLA-B*15:01,0,YYAMYREISTNTYESNLYLRYDSYTWAEWAYLWY
4,CLLTPGVQG,9,HLA-A*03:01,0,YFAMYQENVAQTDVDTLYIIYRDYTWAELAYTWY


In [18]:
gs = df.groupby(['peptide'])

In [19]:
group1 = []
group2 = []
n_error = 0
random.seed(42)
with tqdm(gs) as pbar:
    for peptide, g_df in pbar:
        length = len(g_df)
        true_sum = (g_df['label'] == 1).sum()
        try:
            if (true_sum == 0 or true_sum == length):
                if random.randint(0,9) == 0:
                    group2.append(g_df)
                else:
                    group1.append(g_df)
            else:
                n = random.randint(0, 1)
                group1.append(g_df[g_df['label'] == n])
                group2.append(g_df[g_df['label'] == int(not n)])
        except:
            print(f'wrong: {peptide}, {g_df}')
            n_error += 1

  0%|          | 0/583866 [00:00<?, ?it/s]

In [21]:
from src.tools.pickle import to_pickle, from_pickle

In [22]:
to_pickle(group1, '../cache/tmp_group1.pkl', force=True)
to_pickle(group2, '../cache/tmp_group2.pkl', force=True)

pickle in to tmp_group1.pkl in force
pickle in to tmp_group2.pkl in force


'tmp_group2.pkl'

In [7]:
group1 = from_pickle('../cache/tmp_group1.pkl')
group2 = from_pickle('../cache/tmp_group2.pkl')

In [20]:
len(group1), len(group2)

(525999, 62549)

In [11]:
group1.to_csv(new_train_path)
group2.to_csv(new_val_path)

In [24]:
from src.tools.pandas import show_ratio
show_ratio(pd.read_csv(new_train_path, index_col=0), 'label')
show_ratio(pd.read_csv(new_val_path, index_col=0), 'label')

标签 0 比例为: 50.02%, 个数为: 323370
标签 1 比例为: 49.98%, 个数为: 323129
标签 1 比例为: 50.17%, 个数为: 36037
标签 0 比例为: 49.83%, 个数为: 35796
